In [ ]:
import json
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime
import pandas as pd

# Initialize best bid and ask values
btc_usd_bid = None
btc_usd_ask = None

# Storage for bid/ask data with timestamps
data_storage = []

# Assuming api key/ secret are pre defined

client = RESTClient(api_key=api_key, api_secret=api_secret)

# Function to fetch market trades for BTC-USD
def get_market_trades(product_id):
    return client.get_market_trades(product_id=product_id, limit=1)

# Product ID for BTC-USD
product_id = "BTC-USD"

# Collect data for 30 mins (1800 seconds) at 1-second intervals
start_time = time.time()
end_time = start_time + 1800  # 5 minutes = 300 seconds

while time.time() < end_time:
    with ThreadPoolExecutor(max_workers=1) as executor:
        # Submit the task to fetch BTC-USD market trades
        futures = {executor.submit(get_market_trades, product_id): product_id}
        
        # Collect results as they are completed
        timestamp = datetime.now()  # Record the current timestamp
        for future in as_completed(futures):
            product_id = futures[future]
            try:
                data = future.result()

                # Check if the response data is a string or dictionary
                if isinstance(data, str):
                    response_data = json.loads(data)
                elif isinstance(data, dict):
                    response_data = data
                else:
                    raise ValueError("Unexpected response format")

                # Extract the best bid and ask values from the response
                best_bid = float(response_data['best_bid'])
                best_ask = float(response_data['best_ask'])

                # Store the best bid/ask values along with the timestamp
                btc_usd_bid = best_bid
                btc_usd_ask = best_ask

                # Store the result in the data storage list
                data_storage.append({
                    "timestamp": timestamp,
                    "product_id": product_id,
                    "best_bid": best_bid,
                    "best_ask": best_ask
                })

                # Print the best bid and ask values with timestamp
                print(f"{timestamp} - {product_id} - Best Bid: {best_bid}, Best Ask: {best_ask}")
                
            except Exception as exc:
                print(f"{product_id} generated an exception: {exc}")

    # Sleep for 1 second before fetching data again
    time.sleep(1)

# Convert the stored data to a Pandas DataFrame for further analysis
df = pd.DataFrame(data_storage)

# Print the DataFrame or save it to a file
print(df)

# Optionally, you can save this data to a CSV file for later use:
df.to_csv('btc_usd_bid_ask_data30mins.csv', index=False)
